In [1]:
import pandas as pd
import numpy as np

In [2]:
# dataload 
!ls underexpose_train
!ls underexpose_test

underexpose_item_feat.csv      underexpose_train_click-4.zip
underexpose_train_click-0.csv  underexpose_train_click-5.zip
underexpose_train_click-1.csv  underexpose_train_click-6.zip
underexpose_train_click-1.zip  underexpose_train_click-7.zip
underexpose_train_click-2.csv  underexpose_train_click-8.zip
underexpose_train_click-2.zip  underexpose_train_click-9.zip
underexpose_train_click-3.zip  underexpose_user_feat.csv
underexpose_test_click-0.csv  underexpose_test_click-5.zip
underexpose_test_click-0.zip  underexpose_test_click-6.zip
underexpose_test_click-1.csv  underexpose_test_click-7.zip
underexpose_test_click-1.zip  underexpose_test_click-8.zip
underexpose_test_click-2.csv  underexpose_test_click-9.zip
underexpose_test_click-2.zip  underexpose_test_qtime-0.csv
underexpose_test_click-3.zip  underexpose_test_qtime-1.csv
underexpose_test_click-4.zip  underexpose_test_qtime-2.csv


In [3]:
########  item data 
data_item = pd.read_pickle('Feature/underexpose_item_feat.pkl')
#######  user data 
data_user = pd.read_csv('underexpose_train/underexpose_user_feat.csv',sep=',',names = ['user_id','user_age_level','user_gender','user_city_level'])
# user_id, user_age_level, user_gender, user_city_level
########  train data 
train_click0 = pd.read_csv('underexpose_train/underexpose_train_click-0.csv',sep=',',names = ['user_id','item_id','time'])
train_click1 = pd.read_csv('underexpose_train/underexpose_train_click-1.csv',sep=',',names = ['user_id','item_id','time'])
train_click2 = pd.read_csv('underexpose_train/underexpose_train_click-2.csv',sep=',',names = ['user_id','item_id','time'])
# user_id,item_id,time
########  test data 
test_click0 = pd.read_csv('underexpose_test/underexpose_test_click-0.csv',sep=',',names = ['user_id','item_id','time'])
test_click1 = pd.read_csv('underexpose_test/underexpose_test_click-1.csv',sep=',',names = ['user_id','item_id','time'])
test_click2 = pd.read_csv('underexpose_test/underexpose_test_click-2.csv',sep=',',names = ['user_id','item_id','time'])
# user_id,item_id,time
test_qtime0 = pd.read_csv('underexpose_test/underexpose_test_qtime-0.csv',sep=',',names = ['user_id','time'])
test_qtime1 = pd.read_csv('underexpose_test/underexpose_test_qtime-1.csv',sep=',',names = ['user_id','time'])
test_qtime2 = pd.read_csv('underexpose_test/underexpose_test_qtime-2.csv',sep=',',names = ['user_id','time'])
# user_id,time
########  merge all the clicked records 
data_click_T = pd.concat([train_click0,train_click1,train_click2,test_click0,test_click1,test_click2],axis = 0).reset_index(drop = True)

In [8]:
print(data_item.shape)
print(data_user.shape)
print(data_click_T.shape)


(108916, 3)
(6789, 4)
(795911, 3)


In [9]:
data_item.head()

,item_id,text_vec,img_vec
0,42844,"[4.514945030212402, -2.383719682693481, 0.5004...","[-2.8722801208496094, 1.4587551355361938, 2.57..."
1,67898,"[-2.0029051303863525, -0.9298805594444276, 0.7...","[-0.07052088528871536, -1.4393335580825806, 0...."
2,66446,"[4.221673011779785, -1.4971394538879397, 1.133...","[-5.18036413192749, -0.38824713230133057, -0.0..."
3,63651,"[2.6579699516296387, -0.941863477230072, 1.121...","[-1.077273964881897, 2.8394529819488525, 1.183..."
4,46824,"[3.192194938659668, -1.9366759061813357, 1.199...","[-3.2601945400238037, 0.4988903403282166, 2.58..."


In [10]:
# find all clicked items of submit user_id (in test_qtime_T)
test_qtime_T = pd.concat([test_qtime0, test_qtime1,test_qtime2], axis=0).reset_index(drop = True)
test_click_T = pd.concat([test_click0, test_click1,test_click2], axis=0).reset_index(drop = True)
train_click_T = pd.concat([train_click0, train_click1,train_click2], axis=0).reset_index(drop = True)
print(test_qtime_T.shape)
print(test_click_T.shape)

print(data_user['user_id'].nunique())
print(train_click_T['user_id'].nunique())
print(test_qtime_T['user_id'].nunique())
print(test_click_T['user_id'].nunique())

(5079, 2)
(68426, 3)
6786
23307
5079
5079


In [11]:
# # check1: if user_id in q_time are in train file? 0. in test file?:all 1663/1663
# user_ii=[]
# for i in test_qtime0['user_id'].unique():
#     if i in train_click0['user_id'].unique(): 
#         user_ii.append(i) 
# len(set(user_ii)) 
# # print(set(user_ii))

In [12]:
# # check2: if user_id in q_time are in data_user file? partly: 386/1663
# user_ii=[]
# for i in test_qtime0['user_id'].unique():
#     if i in data_user['user_id'].unique():
#         user_ii.append(i) 
# len(set(user_ii)) # 4016 not in data_user,it means no age gender and city,only has clicked data in test file
# # print(set(user_ii))

In [13]:
# # check3: if user&item in test0 file are in train1 file? 
# test_click1['user_item_id']=test_click1['user_id'].map(str)+'&'+test_click1['item_id'].map(str)
# train_click2['user_item_id']=train_click2['user_id'].map(str)+'&'+train_click2['item_id'].map(str)
# # print(test_click1['user_item_id'].nunique()) #24465
# # print(train_click2['user_item_id'].nunique()) #243569
# user_item_ii=[]
# for i in test_click1['user_item_id'].unique():
#     if i in train_click2['user_item_id'].unique():
#         user_item_ii.append(i) 
# len(set(user_item_ii)) # 4016 not in data_user,it means no age gender and city,only has clicked data in test file
# # print(set(user_ii))

In [14]:
# # check4: if user in test0 file are in train1 file? partly 
# user_ii=[]
# for i in test_click0['user_id'].unique():
#     if i in train_click1['user_id'].unique():
#         user_ii.append(i) 
# len(set(user_ii)) 

In [15]:
# summary : qtime0 user_id all in test0 file,not in train0 file,but in train1 file.

In [16]:
# 1. for each user, clicked item list, item_cnt
data_click_T_user = data_click_T[['user_id','item_id']].groupby('user_id',as_index= False)['item_id'].agg({'item_dis':'unique','item_cnt':'nunique'})
print(data_click_T_user.shape)
print(data_click_T_user.head())

(23816, 3)
   user_id                                           item_dis  item_cnt
0        1  [47611, 76240, 78142, 89568, 97795, 78380, 178...        25
1        2  [7053, 7745, 4983, 23824, 104635, 4168, 8481, ...        33
2        3  [41811, 68822, 114094, 64509, 96045, 2396, 526...        14
3        4  [13283, 90818, 35419, 6609, 8291, 64404, 18883...         8
4        6  [92160, 60291, 77900, 20582, 44128, 40062, 409...        11


In [17]:
# 2. for each item, click_cnt rank
data_click_T_item = data_click_T.groupby('item_id',as_index= False)['user_id'].agg({'user_dis':'unique','click_cnt':'count'})
print(data_click_T_item.head())
data_click_T_item_rank = data_click_T_item.sort_values(by = 'click_cnt',axis = 0,ascending = False)['item_id'].tolist()
data_click_T_item_rank[0:4]

   item_id                                           user_dis  click_cnt
0        1                        [10088, 1489, 19312, 13420]          4
1        2                       [31391, 19402, 10765, 18252]          4
2        3  [5206, 24081, 18247, 8022, 2329, 5788, 10370, ...         22
3        4                         [29720, 34045, 8248, 6181]          4
4        9  [16439, 25683, 31880, 30103, 30007, 32015, 347...         30


[52766, 111225, 2420, 114125]

In [19]:
# 3.Phrase T=2 :for submit user,find last click item,and then find users clicked the item next item to click 
    # submit user                        in Phrase T=2 
    # last click item                    in Phrase T=2  in text file
    # other users clicked the item       in Phrase T=2  in train/text file
    # next item by those users by time       in Phrase T=2  in train/text file    
def next_item_by(user_list):
    item_seq={}
    for user_id in user_list:
        temp = data_click_T[data_click_T.user_id == user_id]
        temp_1 = temp.sort_values(by = 'time',axis = 0,ascending = False)
        c = temp_1['item_id'].tolist()
        for index,item in enumerate(c):
            if index ==len(c)-1:break
            if item not in item_seq:
                item_seq[item] = {}
                if c[index+1] not in item_seq[item]:
                    item_seq[item][c[index+1]] = 1
                else:
                    item_seq[item][c[index+1]] += 1
            else:    
                if c[index+1] not in item_seq[item]:
                    item_seq[item][c[index+1]] = 1
                else:
                    item_seq[item][c[index+1]] += 1
    return item_seq

# all the item in data with the next item and item_seq
item_seq = next_item_by(data_click_T['user_id'].unique())
#

In [20]:
# for submit user,find last click item
def last_item(user_list):
    user_lastitem={}
    for user_id in user_list:
        temp =test_click_T[test_click_T.user_id == user_id]
        temp_1 = temp.sort_values(by = 'time',axis = 0,ascending = False).iloc[0:1,1:2].values[0][0]
        user_lastitem[user_id]= temp_1
    return user_lastitem

user_lastitem = last_item(test_click_T.user_id.tolist())

In [21]:
user_nextitem_dict= {}
for user_id in user_lastitem.keys():
    item_id = user_lastitem[user_id]
    next_item_dict = item_seq[item_id]
    temp =sorted(next_item_dict.items(),key = lambda x:x[1],reverse = True)
    next_item_list =[]
    for i in range(len(temp)):
        next_item_list.append(temp[i][0])
    user_nextitem_dict[user_id]= next_item_list

In [22]:
def deal(x):
    y=[]
    if len(user_nextitem_dict[x]) >=50:
        y = user_nextitem_dict[x][0:50]
    else:
        y=user_nextitem_dict[x]
        for i in data_click_T_item_rank:
            if i in y:continue
            y.append(i)
            if len(y)>=50:
                break
    return y
    
test_qtime_T['candidate'] = test_qtime_T['user_id'].apply(lambda x :deal(x))

In [23]:
test_click_T.head()

,user_id,item_id,time
0,1133,221,0.983812
1,17864,253,0.983783
2,6941,309,0.983785
3,34089,358,0.983781
4,21659,536,0.983793


In [24]:
for i in range(50):
    if i < 9:
        col = 'item_id_0'+str(i+1)
    else:
        col = 'item_id_'+str(i+1)
    test_qtime_T[col] = test_qtime_T['candidate'].apply(lambda x: x[i])    

In [25]:
test_qtime_T.head()
test_qtime_T = test_qtime_T.drop(columns=['time','candidate'])

In [26]:
test_qtime_T.shape

(5079, 51)

In [27]:
#test_qtime2.drop_duplicates(subset = 'user_id',inplace = True)
test_qtime_T.to_csv('Result/underexpose_submit_T.csv',sep=',',index = False,header = False)